# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [32]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [2]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [7]:
df.head(20)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [3]:
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [4]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [25]:
click_ids = set(df[df.action=='click']['id'].unique())
view_ids = set(df[df.action=='view']['id'].unique())
click_but_not_view_list = [id for id in click_ids if id not in view_ids]
click_but_not_view_list

[]

In [15]:
df['group'].value_counts()


control       4264
experiment    3924
Name: group, dtype: int64

In [16]:
4264+3924

8188

In [31]:
control_ids = set(df[df.group=='control']['id'].unique())
experiment_ids = set(df[df.group=='experiment']['id'].unique())
control_and_experiment = [id for id in control_ids if id is in experiment_ids]

SyntaxError: invalid syntax (<ipython-input-31-8c2a12c222ea>, line 3)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [48]:
experiment_clicks = df[(df['group']=='experiment') & (df['action']=='click')]
experiment_clicks


,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8117,2017-01-17 09:27:37.808307,891635,experiment,click
8133,2017-01-17 15:09:17.533736,865656,experiment,click
8152,2017-01-17 20:14:12.483794,413359,experiment,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click


In [56]:

experiment_views = len(df.loc[df['group']=='experiment'])-len(df[(df['group']=='experiment') & (df['action']=='click')])
experiment_clicks = len(df[(df['group']=='experiment') & (df['action']=='click')])

control_views = len(df.loc[df['group']=='control'])-len(df[(df['group']=='control') & (df['action']=='click')])
control_clicks = len(df[(df['group']=='control') & (df['action']=='click')])

observations = np.array([experiment_clicks, experiment_views - experiment_clicks])
expectations = np.array([control_clicks, control_views - control_clicks])

#Your code here
result = stats.chisquare(f_obs=observations, f_exp=expectations)
result

Power_divergenceResult(statistic=45.943834048640916, pvalue=1.2169234221091101e-11)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [ ]:
#Your code here

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.